# excels2vensim

## Description

### Functionality
1. Reads excel files with any structure
2. Assigns cellrange names (unique per sheet and file)
3. Generates Vensim code

### Advantages
* It does not require any changes in the structure of the Excel files
* It can work with data spread in different tabs and/or files
* No limit on the number of dimensions (VDF only reads up to 8)
* Easy to change the order of the dimensions in the Vensim file in any time
* Compatible with Python translation
* Compatible with version control
* Able to work with several variables at a time
* It automatically adds * at the end of Constant's cellrange name when transposition is needed
* Reusable

### Disadvantages
* Requires installing Python
* Requires manual mapping (as VDFs)


## Notes
This version is only a proposal. If it is accepted, several improvements can be done to make it more solid and easy to use:
* A simple GUI can be developed to make the usage mutch simpler. This will avoid forcing the user to create json files as shown in the examples.
* Some errors and warnings should be given when trying to write duplicated cellrange names.
* Support for DATA and LOOKUPS is not yet implemented, but it should be relatively easy.
* Currently the subscript range values are hardcoded, this must be changed in the future using the Vensim parser to automatically detect the model subscripts.

## Example 1: Reading simple vectors

Let's suppose we have the **source** subscript defined in Vensim model with values Gas, Oil, and Coal, i.e.:

```
source:
    Gas, Oil, Coal 
    ~
    ~ 
    |
```

This can be given in the excel file in two ways:
1. Column vector (*B18:B20*) 
2. Row vector (*A24:C24*).

In the first case (*B18:B20*) the dimension **source** is divided along the **row**s with step **1**, i.e., each row has a new value of **source** dimension. In the second case (*A24:C24*) the dimension *source* is divided along the **col**s with step **1**, i.e., each column has a new value of *source* dimension.

We have the data in *Region1* sheet and *inputs.xlsx* file:

![title](figures/excel1.png)

Let's suppose we want to retrieve the information for reading the variable **q_row** from *B18:B20* and **q_col** from *A24:C24*. Then we need to create a file with the information of the variable with the following shape (this would be avoided with a GUI)

```
{
'q_row': {
    'dims': ['source'],                      # dimensions names list
    'cell': 'A24',                           # reference cell, first cell with data values
    'file': 'example_data/inputs.xlsx',      # file name
    'sheet': 'Region1',                      # sheet name
    'description': 'This is my variable q_row read from B18:B20.',   # description
    'units': 'TWh',                          # units
    'dimensions':{                           # read along information
        'source': ['col', 1]                 # dimension name: [read_along, step]
        }
    },
'q_col': {
    'dims': ['source'],                      # dimensions names list
    'cell': 'B18',                           # reference cell, first cell with data values
    'file': 'example_data/inputs.xlsx',      # file name
    'sheet': 'Region1',                      # sheet name
    'description': 'This is my variable q_col read from A24:C24.',   # description
    'units': 'TWh',                          # units
    'dimensions':{                           # read along information
        'source': ['row', 1]                 # dimension name: [read_along, step]
        }
    }
}
```

This information was already saved in the *example1.json* file. should be run with the following command:
```
from run import constants
constants('example_data/example1.json')
```
To run it in a command line (no need to install jupyter notebook):
```
python run.py example_data/example1.json
```
after runing, the Vensim instructions to copy in the *text mode* of the Vensim file will be given and the cell range name will be created in the Excel file.

### Note
For running in Vensim before the run the Excel file must be open with Excel and save it.

In [2]:
from run import constants
constants('example_data/example1.json')


q_row[source]=
	GET_DIRECT_CONSTANTS('example_data/inputs.xlsx', 'Region1', 'q_row')
	~	TWh
	~	This is my variable q_row read from B18:B20.
	|

q_col[source]=
	GET_DIRECT_CONSTANTS('example_data/inputs.xlsx', 'Region1', 'q_col*')
	~	TWh
	~	This is my variable q_col read from A24:C24.
	|


## Example 2: Reading 3 dims matrix

Lets suppose we have the **source** subscript define in vensim with values Gas, Oil, and Coal; **sector** with A, B, and C; **out** with Elec, Heat, Solid, and Liquid; i.e.:

```
out:
        Elec, Heat, Solid, Liquid
        ~
        ~               |

sector:
        A, B, C, D
        ~
        ~               |

source:
        Gas, Oil, Coal
        ~
        ~               |

```

and the following data in *Region1* sheet and *inputs.xlsx* file:

![title](figures/excel2.png)

We want to read `share_energy_region1[sector, source, out]`

Now the dimension **sector** is along **col**s with step **1**, as each column the value of *sector* changes; the dimension **source** is along **row**s with step **1**, as each row the value of *source* changes; the dimension **out** is along **row**s with step **3**, as each 3 rows the value of *out* changes.

We can create Vensim equations and save the cellrange names with the file *example2.json* which has the folowing information
```
{
'share_energy_region1': {
    'dims': ['sector', 'source', 'out'],
    'cell': 'C4',
    'file': 'example_data/inputs.xlsx',
    'sheet': 'Region1',
    'description': 'This is my second variable.',
    'units': 'dmnl',
    'dimensions':{
        'sector': ['col', 1],
        'source': ['row', 1],
        'out': ['row', 3]
        }
    }
}
```

In [3]:
constants('example_data/example2.json')


share_energy_region1[sector, source, Elec]=
	GET_DIRECT_CONSTANTS('example_data/inputs.xlsx', 'Region1', 'share_energy_region1_1*') ~~|
share_energy_region1[sector, source, Heat]=
	GET_DIRECT_CONSTANTS('example_data/inputs.xlsx', 'Region1', 'share_energy_region1_2*') ~~|
share_energy_region1[sector, source, Solid]=
	GET_DIRECT_CONSTANTS('example_data/inputs.xlsx', 'Region1', 'share_energy_region1_3*') ~~|
share_energy_region1[sector, source, Liquid]=
	GET_DIRECT_CONSTANTS('example_data/inputs.xlsx', 'Region1', 'share_energy_region1_4*')
	~	dmnl
	~	This is my second variable.
	|


## Example 3: Reading 4 dims matrix along sheets

Lets suppose we have the **source** subscript define in vensim with values Gas, Oil, and Coal; **sector** with A, B, and C; **out** with Elec, Heat, Solid, and Liquid; **regions** with Region1, Region2, Region3, and Region4 i.e.:

```
out:
        Elec, Heat, Solid, Liquid
        ~
        ~               |

sector:
        A, B, C, D
        ~
        ~               |

source:
        Gas, Oil, Coal
        ~
        ~               |

region:
	Region1, Region2, Region3, Region4
	~	
	~		|

```

and the following data in sheets Region1, Region2, Region3, and Region4 and *inputs.xlsx* file:

![title](figures/excel3.png)

We want to read `share_energy[source, sector, region, out]`

Now the dimension **sector**,  **source**, and **out** are defined as in Example 2. The dimension **regions** is defined along **sheet**s for sheets **Region1**, **Region2**, **Region3**, **Region4**.

We can create Vensim equations and save the cellrange names with the file *example3.json* which has the folowing information
```
{
'share_energy': {
    'dims': ['source', 'sector', 'region', 'out'],
    'cell': 'C4',
    'file': 'example_data/inputs.xlsx',
    'description': 'This is my variable.',
    'units': 'dmnl',
    'dimensions':{
        'sector': ['col', 1],
        'region': ['sheet', ['Region1', 'Region2', 'Region3', 'Region4']],
        'source': ['row', 1],
        'out': ['row', 3]
        }
    }
}
```

In [4]:
constants('example_data/example3.json')


share_energy[source, sector, Region1, Elec]=
	GET_DIRECT_CONSTANTS('example_data/inputs.xlsx', 'Region1', 'share_energy_1') ~~|
share_energy[source, sector, Region1, Heat]=
	GET_DIRECT_CONSTANTS('example_data/inputs.xlsx', 'Region1', 'share_energy_2') ~~|
share_energy[source, sector, Region1, Solid]=
	GET_DIRECT_CONSTANTS('example_data/inputs.xlsx', 'Region1', 'share_energy_3') ~~|
share_energy[source, sector, Region1, Liquid]=
	GET_DIRECT_CONSTANTS('example_data/inputs.xlsx', 'Region1', 'share_energy_4') ~~|
share_energy[source, sector, Region2, Elec]=
	GET_DIRECT_CONSTANTS('example_data/inputs.xlsx', 'Region2', 'share_energy_1') ~~|
share_energy[source, sector, Region2, Heat]=
	GET_DIRECT_CONSTANTS('example_data/inputs.xlsx', 'Region2', 'share_energy_2') ~~|
share_energy[source, sector, Region2, Solid]=
	GET_DIRECT_CONSTANTS('example_data/inputs.xlsx', 'Region2', 'share_energy_3') ~~|
share_energy[source, sector, Region2, Liquid]=
	GET_DIRECT_CONSTANTS('example_data/inputs.xlsx', 'R